# Convolutional Neuronal Network on MNIST

In [2]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.models import model_from_json

In [3]:
batch_size = 128
num_classes = 10
epochs = 12

In [4]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [5]:
from pathlib import Path
model = Path("./Data/modelcnn.json")
if model.is_file():
   # load json and create model
    json_file = open('./Data/modelcnn.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights("./Data/modelcnn.h5")
    print("Loaded model from disk")
else:
    modelcnn = Sequential()
    modelcnn.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    modelcnn.add(Conv2D(64, (3, 3), activation='relu'))
    modelcnn.add(MaxPooling2D(pool_size=(2, 2)))
    modelcnn.add(Dropout(0.25))
    modelcnn.add(Flatten())
    modelcnn.add(Dense(128, activation='relu'))
    modelcnn.add(Dropout(0.5))
    modelcnn.add(Dense(num_classes, activation='softmax'))

In [6]:
modelcnn.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [7]:
modelcnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)


Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 158s 3ms/step - loss: 0.2528 - accuracy: 0.9220 - val_loss: 0.0549 - val_accuracy: 0.9816
Epoch 2/12
60000/60000 [==============================] - 158s 3ms/step - loss: 0.0848 - accuracy: 0.9752 - val_loss: 0.0417 - val_accuracy: 0.9860
Epoch 3/12
60000/60000 [==============================] - 157s 3ms/step - loss: 0.0642 - accuracy: 0.9805 - val_loss: 0.0355 - val_accuracy: 0.9877
Epoch 4/12
60000/60000 [==============================] - 163s 3ms/step - loss: 0.0539 - accuracy: 0.9835 - val_loss: 0.0282 - val_accuracy: 0.9906
Epoch 5/12
60000/60000 [==============================] - 165s 3ms/step - loss: 0.0459 - accuracy: 0.9862 - val_loss: 0.0291 - val_accuracy: 0.9899
Epoch 6/12
60000/60000 [==============================] - 167s 3ms/step - loss: 0.0404 - accuracy: 0.9879 - val_loss: 0.0273 - val_accuracy: 0.9908
Epoch 7/12
60000/60000 [==============================] - 167

AttributeError: 'WindowsPath' object has no attribute 'evaluate'

In [ ]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Save trained Model to disk via JSON
model_json = modelcnn.to_json()
with open("./Data/modelcnn.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("./Data/modelcnn.h5")
print("Saved model to disk")